In [2]:
# Setup stuff goes here
import utils

utils.setupOpenAIAPI()

# Text to Audo (TTA) Demostration

Converts text to a mp3 file with OpenAI's TTA model

In [ ]:
audioFileOutput = "audio_ouput.mp3"
text = "Hello World! Hear me speak!"

utils.textToAudioFile(text, audioFileOutput) # Convert example text to audio file as test

# Audio to Text (ATT) Demostration

Converts Audio file to text(ATT) 

In [ ]:
audioFileInput = "Audio Input Test.m4a" 

transcription = utils.audioToText(audioFileInput)
print(transcription)

# First Echo Attempt
Successfully detects human voice, starts recording when human speaks, stops recording when human speaks and then converts human speech to text and then text to audio and plays back audio the human spoke in OpenAI's TTA voice.

Not real time or useful in practical as only when finish recording does it translate to text and text to audio and only when audio is finished writting to file does it play back. this means that it is so slow and this is the subject of the next improvement

In [3]:
import os
import sounddevice as sd
import scipy.io.wavfile as wav
import numpy as np
import webrtcvad
from pydub import AudioSegment
from pydub.playback import play
import collections
import time

# Define audio parameters
fs = 16000 
frame_duration_ms = 30 

def record_audio_vad(filename):
    vad = webrtcvad.Vad(1)
    num_padding_frames = 10 
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    triggered = False
    voiced_frames = []
    print("Recording... Speak now.")

    silence_threshold_seconds = 1.5 
    last_voice_timestamp = time.time()

    with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
        while True:
            audio_chunk, overflowed = stream.read(int(fs * frame_duration_ms / 1000))
            is_speech = vad.is_speech(audio_chunk.tobytes(), fs)

            if is_speech:
                last_voice_timestamp = time.time()

            if not triggered:
                ring_buffer.append((audio_chunk, is_speech))
                num_voiced = len([f for f, speech in ring_buffer if speech])
                if num_voiced > 0.9 * ring_buffer.maxlen:
                    triggered = True
                    print("Recording started.")
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(audio_chunk)
                ring_buffer.append((audio_chunk, is_speech))
                num_unvoiced = len([f for f, speech in ring_buffer if not speech])
                
                if num_unvoiced > 0.9 * (ring_buffer.maxlen) and (time.time() - last_voice_timestamp > silence_threshold_seconds):
                    print("Recording stopped.")
                    break

    audio_data = np.concatenate(voiced_frames, axis=0)
    wav.write(filename, fs, audio_data)
    print("Recording complete.")

def transcribe_audio(filename):
    return utils.audioToText(filename) 

def synthesize_speech(text, output_filename):
    utils.textToAudioFile(text, output_filename) 

def play_audio(filename):
    audio = AudioSegment.from_mp3(filename)
    play(audio)

input_audio_file = "input5.wav"
output_audio_file = "output5.mp3"

record_audio_vad(input_audio_file)
transcription = transcribe_audio(input_audio_file)
print("Transcription:", transcription)

synthesize_speech(transcription, output_audio_file)
play_audio(output_audio_file)


Recording... Speak now.
Recording started.
Recording stopped.
Recording complete.
Transcription: This is the test.


# Second Echo Attempt

In [ ]:
import os
import sounddevice as sd
import scipy.io.wavfile as wav
import numpy as np
import webrtcvad
from pydub import AudioSegment
from pydub.playback import play
import collections
import time
import threading
import queue
import io  # For in-memory audio handling


fs = 16000
frame_duration_ms = 250  # Smaller chunk size
audio_queue = queue.Queue()
transcription_queue = queue.Queue()
speech_queue = queue.Queue()

# --- VAD and Audio Chunking ---
def record_audio_chunks():
    vad = webrtcvad.Vad(1) 
    # ... (Adjust VAD parameters for responsiveness) 
    with sd.InputStream(samplerate=fs, channels=1, dtype='int16') as stream:
        while True:
            # ... (Record audio chunk)
            audio_queue.put(audio_chunk)

# --- Asynchronous Transcription ---
def transcribe_worker():
    while True:
        audio_chunk = audio_queue.get()
        # ... (Convert audio_chunk to bytes-like object) 
        with io.BytesIO() as audio_buffer:
            wav.write(audio_buffer, fs, audio_chunk)
            audio_buffer.seek(0) 
            transcription = utils.audioToText(audio_buffer) 
        transcription_queue.put(transcription)
        audio_queue.task_done()

# --- Asynchronous Speech Synthesis ---
def synthesize_worker():
    while True:
        text = transcription_queue.get()
        with io.BytesIO() as speech_buffer:
            utils.textToAudioFile(text, speech_buffer)
            speech_buffer.seek(0)
            speech_data = speech_buffer.read()
        speech_queue.put(speech_data)
        transcription_queue.task_done()

# --- Audio Playback --- 
def play_audio_worker():
    while True:
        speech_data = speech_queue.get()
        audio = AudioSegment.from_mp3(io.BytesIO(speech_data)) 
        play(audio)
        speech_queue.task_done() 

# --- Main Execution ---
if __name__ == "__main__":
    # ... (Set up your OpenAI API client)

    # Start worker threads
    threading.Thread(target=record_audio_chunks, daemon=True).start()
    threading.Thread(target=transcribe_worker, daemon=True).start()
    threading.Thread(target=synthesize_worker, daemon=True).start()
    threading.Thread(target=play_audio_worker, daemon=True).start()

    # Keep the main thread alive
    while True:
        time.sleep(1)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/ursusprizlius/.pyenv/versions/3.9.6/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/ursusprizlius/Documents/Projects/audio-to-text-client/.venv/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/ursusprizlius/.pyenv/versions/3.9.6/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/qn/g265ll392sz062bdvw23pz680000gn/T/ipykernel_15697/1540040607.py", line 28, in record_audio_chunks
NameError: name 'audio_chunk' is not defined
